# A hands-on guide to LLM-based AI Agents
# Exercise Block 2
#### ML Prague 2025
#### Philipp Wendland <pwendland@deloitte.de>
This notebook contains exercises for the workshop at ML Prague 2025. The goal is to implement an AI Agent that can assist you in exploring a new city and planning various trips according to your preferences.
***
###### Sources: 
- https://huggingface.co/blog/smolagents
- https://huggingface.co/docs/smolagents/index
- https://github.com/anthropics/anthropic-cookbook/
- https://platform.openai.com/docs/overview
***

## Exercise 3b: Tool Calling with smolagents
Create your first "agent" using smolagents.

- [ ] Examine the simple way to setup a CodeAgent in smolagents
- [ ] Add the implemented get_weather tool and the DuckDuckGoSearchTool to a Code Agent and let it create an intinerary as before. Run the agent several times and adjust the prompt for the best results. Notice potential issues with setting up an agent like this.

In [ ]:
from smolagents import CodeAgent, LiteLLMModel
import os

model = LiteLLMModel(model_id="openai/gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
# Just a showcase on how to create an agent in smolagents - not a particularly good use case
agent = CodeAgent(tools=[], model=model)
agent.run("Act as a professional travel planner. I'm staying in Prague for the Machine Learning Prague conference.\
    During the day I will attend seminars and talks, but in the evening I'm mostly free. Create a list of items that\
    could potentially be interesting. Be creative")

In [ ]:
# Examine the system prompt
print(agent.prompt_templates["system_prompt"]) 

In [ ]:
# open-meteo docs: https://open-meteo.com/en/docs
import requests
import json
from smolagents import CodeAgent, LiteLLMModel, DuckDuckGoSearchTool, tool

# Optional: Include more weather information into the tool - make sure to describe them appropriately 
@tool
def get_weather(latitude: float, longitude: float) -> dict:
    """
        Obtains the current weather based on a given location
        Args:
            latitude: float of latitude of the given location
            longitude: float of longitude of the given location
        Returns:
            dict containing
                "current_temp": current temperature in celsius
                "cloud_cover": current cloud cover in percent
                "percipitation": current percipitation in mm
    """
    try: 
        response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,cloud_cover,precipitation&hourly=temperature_2m,precipitation_probability,cloud_cover")
    except: 
        return "Weather data not available"
    response_data = response.json()
    return {
        "current_temp": response_data['current']['temperature_2m'],
        "cloud_cover": response_data['current']['cloud_cover'],
        "precipitation": response_data['current']['precipitation']
    }

model = LiteLLMModel(model_id="openai/gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

agent = CodeAgent(tools=[DuckDuckGoSearchTool(), get_weather], model=model, additional_authorized_imports=['requests'])

prompt = """First, identify the current weather in prague. Then act as a professional travel planner. I'm staying in Prague for the Machine Learning Prague conference.
    During the day I will attend seminars and talks, but in the evening I'm mostly free. Create a list of items that
    could potentially be interesting, by searching the internet. Be creative. Lastly, Plan an itinerary for must-see things in Prague for Monday and Tuesday night."""

agent.run(prompt)


## Exercise 4: Multi-agent systems
As you saw in the last exercise, giving one agent a longer list of tools usually does not yield the best results. It is best practice to use agents to solve specific tasks (e.g., one agent to perform web-search and another agent to obtain weather information). In this exercise we will create a multi-agent system
- [ ] Examine the multi-agent system consisting of two `ToolCallingAgents` and one manager agent (a `CodeAgent`)
- [ ] Add an additional agent to the multi-agent system. First, implement an a new tool of your choice (e.g., image generation to create a postcard , tourist summary of famous sights, connect to the GoogleMaps API to plan routes between sights) 

In [ ]:
# To make searching the web a little more interesting we give the agent a tool to access webpages 
# Reference: https://huggingface.co/docs/smolagents/en/examples/multiagents#-create-a-web-search-tool

import re
import requests
from markdownify import markdownify
from requests.exceptions import RequestException
from smolagents import tool


@tool
def visit_webpage(url: str) -> str:
    """
    Visits a webpage at the given URL and returns its content as a markdown string.

    Args:
        url: The URL of the webpage to visit.

    Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Convert the HTML content to Markdown
        markdown_content = markdownify(response.text).strip()

        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

## Setup a Multi-Agent system

In [ ]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    DuckDuckGoSearchTool,
    LiteLLMModel,
)

model = LiteLLMModel(model_id="openai/gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

# Defien a new ToolCallingAgent using the get_weather tool from before
weather_agent = ToolCallingAgent(
    tools=[get_weather],
    model=model,
    name="get_weather_agent",
    description="Returns weather data given latitidue and longitude coordinates"
)

# Define a web search agent
web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), visit_webpage],
    model=model,
    max_steps=10,
    name="web_search_agent",
    description="Runs web searches for you.",
)



manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[web_agent, weather_agent],
    additional_authorized_imports=[],
)

prompt = prompt = """First, identify the current weather in prague. Then act as a professional travel planner. I'm staying in Prague for the Machine Learning Prague conference.
    During the day I will attend seminars and talks, but in the evening I'm mostly free. Create a list of items that
    could potentially be interesting, by searching the internet. Be creative. Lastly, Plan an itinerary for must-see things in Prague for Monday and Tuesday night."""

result = manager_agent.run(prompt)


## Example implementation of a new tool

In [ ]:
import json
import urllib.parse

@tool
def create_geojson_link(points: list) -> str:
    """
    Expects a list of points of interest with name and coordinates and returns a link to a map where all points are market.

    Args:
        points: a list of dictionaries with the following format
        [{"name": "Name of point of interest", "latitude": float, "longitude": float},
         {"name": "Name of point of interest 2", "latitude": float, "longitude": float},
        ]

    Returns:
        A url (string) displaying a map with all points visually marked.
    """
    if not points:
        return "No points provided."

    features = []
    for point in points:
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [point['longitude'], point['latitude']]
            },
            "properties": {
                "name": point['name']
            }
        }
        features.append(feature)

    geojson_data = {
        "type": "FeatureCollection",
        "features": features
    }

    # Encode the GeoJSON data as a URL parameter
    geojson_string = json.dumps(geojson_data)
    encoded_geojson = urllib.parse.quote(geojson_string)

    # Construct the geojson.io URL
    geojson_io_url = f"https://geojson.io/#data=data:application/json,{encoded_geojson}"

    return geojson_io_url


## New multi-agent with the additional tool

In [ ]:
map_agent = 
    # TODO: Define the map_agent with the create_geojson_link tool

manager_agent_map = 
    # TODO Define the manager agent, include web_agent, the weather_agent and the new map_agent


# TODO: Adjust the prompt to your preferences
prompt = prompt = """Identify the current weather in prague. Then act as a professional travel planner. I'm staying in Prague for the Machine Learning Prague conference.
    During the day I will attend seminars and talks, but in the evening I'm mostly free. Research places that could be interesting.
    Be creative. Lastly, Plan an itinerary for must-see things in Prague for Monday and Tuesday night.
    Output both the exact itinerary and a map where the stops are highlighed, obtain the coordinates of the stops and use the map_agent to create a visualized map."""

result = manager_agent_map.run(prompt)

## Exercise 4 (continued)

Implement a new tool and create a new multi-agent system using also the previously implemented tools. Ideas for additional tools: 
- Tool that returns coordinates of sights / restaurants (to elimiate the need for web-searches of the agent)
- Tool that checks whether the selected sights are accessible
- Tool that generates a custom postcard based on the visited sights

Hints: https://huggingface.co/docs/smolagents/en/tutorials/tools

In [ ]:
@tool
def new_tool(input_parameter: str) -> str: 
    # TODO: Define new tool 


new_tool_calling_agent =
    # TODO: Define the agent

new_manager_agent = 
    # TODO: Define the manager agent with previous tools and the new tool

# TODO: Run the agent
new_manager_agent.run(prompt)

## Further Reading
- https://huggingface.co/docs/smolagents/en/tutorials/tools
- https://huggingface.co/blog/smolagents
- https://github.com/huggingface/smolagents/blob/main/src/smolagents/vision_web_browser.py
- https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents
- https://huggingface.co/docs/hub/en/spaces-sdks-gradio